<a href="https://colab.research.google.com/github/NeuralNetworkNecromancer/Alrik-Abenteurer/blob/main/Rulebook_chunk_automation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
####INITIATE####
!pip install qdrant-client openai google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client requests PyPDF2 pytz

# utils
import os
import json
import openai
from datetime import datetime
from IPython.display import display, clear_output, Markdown
from google.colab import drive, files, auth
import pytz

#data loading packages
from google.colab import drive, auth
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
import google.auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import PyPDF2
import re
from google.colab import userdata

drive.mount('/content/drive')

# Specify folder and file details
folder_dir = f'/content/drive/MyDrive/<your file path>'
app_dir = f'{folder_dir}/App'
targeted_folder = f'{folder_dir}/Rules'


# Setup Openai
OPENAI_API_KEY = userdata.get('openai')
openai.api_key = OPENAI_API_KEY

# OAuth2.0 Authentication
SCOPES = ['https://www.googleapis.com/auth/drive']

# If modifying these SCOPES, delete the file token.json.
creds = None
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        auth.authenticate_user()
        creds, _ = google.auth.default()

# Build the Drive and Docs services
drive_service = build('drive', 'v3', credentials=creds)
docs_service = build('docs', 'v1', credentials=creds)
# Build the Google Sheets API client.
sheets_service = build('sheets', 'v4', credentials=creds)

clear_output()

In [ ]:
#It seperates files by headers into single docs for upsertion. It could be used as a foundation to build an automation that chunks rulebooks in a meaningful way. However it still is uncomplete as it carries over some letters and doesnt properly process the last page (index related issues)

def get_titles_and_page_numbers(doc):
    content = doc.get('body', {}).get('content', [])
    page_count = 0
    in_page = False
    titles = []

    for element in content:
        text_run = element.get('paragraph', {}).get('elements', [{}])[0].get('textRun', {})
        style = text_run.get('textStyle', {})
        font_size = style.get('fontSize', {}).get('magnitude', None)

        # Check for titles based on font size
        if font_size == 14 and not in_page:
            titles.append({"title": text_run.get('content', '').strip(), "page": page_count + 1})
            in_page = True
            page_count += 1
        # End current page when a newline character is found after a font size 14 line
        elif in_page and '\n' in text_run.get('content', ''):
            in_page = False

    return titles, page_count


# Function to calculate the indices for each page based on font size 14 titles
def calculate_page_indices(doc, titles):
    content = doc.get('body', {}).get('content', [])
    page_indices = []
    current_start_index = None
    current_page = 0
    in_page = False

    for element in content:
        text_run = element.get('paragraph', {}).get('elements', [{}])[0].get('textRun', {})
        style = text_run.get('textStyle', {})
        font_size = style.get('fontSize', {}).get('magnitude', None)

        if font_size == 14 and not in_page:
            if current_start_index is not None:
                page_indices.append({'start': current_start_index, 'end': element.get('startIndex')})
            current_start_index = element.get('startIndex')
            current_page += 1
            in_page = True
        elif in_page and '\n' in text_run.get('content', ''):
            in_page = False

    if in_page:
        page_indices.append({'start': current_start_index, 'end': len(doc['body']['content'])})

    for title_info in titles:
        page_number = title_info['page']
        if page_number <= len(page_indices):
            title_info['indices'] = page_indices[page_number - 1]
            title_info['pages_before'] = page_number - 1
            title_info['pages_after'] = len(page_indices) - page_number

    return titles

def list_folders_and_docs_in_directory(directory_path):
    folder_names = []
    directory_parts = directory_path.split('/')
    current_folder_id = None

    for part in directory_parts:
        if not part:
            continue
        if current_folder_id:
            query = f"name='{part}' and '{current_folder_id}' in parents"
        else:
            query = f"name='{part}'"

        results = drive_service.files().list(q=query, fields="files(id, name, mimeType)").execute()
        for item in results.get('files', []):
            if item['mimeType'] == 'application/vnd.google-apps.folder':
                current_folder_id = item['id']
                break

    folder_results = drive_service.files().list(
        q=f"'{current_folder_id}' in parents and mimeType='application/vnd.google-apps.folder'",
        fields="files(id, name)"
    ).execute()

    for folder in folder_results.get('files', []):
        if folder['name'] == "":
            docs_results = drive_service.files().list(
                q=f"'{folder['id']}' in parents and mimeType='application/vnd.google-apps.document'",
                fields="files(id, name)"
            ).execute()

            for doc_item in docs_results.get('files', []):
                doc = docs_service.documents().get(documentId=doc_item.get('id')).execute()
                titles, number_of_pages = get_titles_and_page_numbers(doc)

                # Process each document for creating one-pagers
                create_one_pagers_for_titles(doc_item.get('id'), titles)

    return folder_names

def calculate_total_content_length(doc):
    content = doc.get('body', {}).get('content', [])
    total_length = 0
    image_value = 1  # Assign 1 for each image

    for element in content:
        # Check for text in paragraphs
        if 'paragraph' in element:
            elements = element.get('paragraph', {}).get('elements', [])
            for elem in elements:
                if 'textRun' in elem:
                    text_run = elem.get('textRun', {})
                    text = text_run.get('content', '')
                    total_length += len(text)
                elif 'inlineObjectElement' in elem:
                    # Count each image as a character
                    total_length += image_value

        # Additional elements like tables, etc., can be handled here

    return total_length

# Use this function within create_one_pagers_for_titles to get the total length
def create_one_pagers_for_titles(document_id, titles):
    original_doc = docs_service.documents().get(documentId=document_id).execute()
    original_content_length = calculate_total_content_length(original_doc)

    # Calculate the page indices for each title
    titles_with_indices = calculate_page_indices(original_doc, titles)

    for title_info in titles_with_indices:
        indices = title_info.get('indices', {})
        pages_before = title_info.get('pages_before')
        pages_after = title_info.get('pages_after') + 1

        # Print the indices for debugging
        print(f"Title: {title_info['title']}, Start Index: {indices.get('start')}, End Index: {indices.get('end')}, Pages Before: {pages_before}, Pages After: {pages_after}, Original Content Length: {original_content_length}")

        # Copy the original document
        copied_doc = drive_service.files().copy(fileId=document_id, body={"name": title_info['title']}).execute()
        copied_doc_id = copied_doc['id']

        # Delete content after the desired page
        if indices.get('end') and indices.get('end') < original_content_length:
            docs_service.documents().batchUpdate(
                documentId=copied_doc_id,
                body={
                    "requests": [
                        {"deleteContentRange": {"range": {"startIndex": indices['end'], "endIndex": original_content_length}}}
                    ]
                }
            ).execute()
            print(f"Deleted after the page: Start Index: {indices['end']}, End Index: {original_content_length}")

        # Delete content before the desired page
        if indices.get('start') and indices.get('start') > 1:
            docs_service.documents().batchUpdate(
                documentId=copied_doc_id,
                body={
                    "requests": [
                        {"deleteContentRange": {"range": {"startIndex": 1, "endIndex": indices['start']}}}
                    ]
                }
            ).execute()
            print(f"Deleted before the page: Start Index: 1, End Index: {indices['start']}")

        print(f"Processed {title_info['title']}")

# Example usage
folder_names = list_folders_and_docs_in_directory(targeted_folder)

In [10]:
#helper to understand the total size of a certain doc
document_id = "15qEieVtUrJ8SzNLqMOAoFUQSt3JuRoFEDgAIziX_dJQ"

def calculate_total_content_length(doc):
    content = doc.get('body', {}).get('content', [])
    total_length = 0
    image_value = 1  # Assign 1 for each image

    for element in content:
        # Check for text in paragraphs
        if 'paragraph' in element:
            elements = element.get('paragraph', {}).get('elements', [])
            for elem in elements:
                if 'textRun' in elem:
                    text_run = elem.get('textRun', {})
                    text = text_run.get('content', '')
                    total_length += len(text)
                elif 'inlineObjectElement' in elem:
                    # Count each image as a character
                    total_length += image_value

        # Additional elements like tables, etc., can be handled here

    return total_length

original_doc = docs_service.documents().get(documentId=document_id).execute()
original_content_length = calculate_total_content_length(original_doc)
print(f"Length: {original_content_length}")


Length: 17537
